# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-07 15:10:00] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-07 15:10:00] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-07 15:10:00] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-07 15:10:02] INFO server_args.py:1615: Attention backend not specified. Use fa3 backend by default.


[2026-01-07 15:10:02] INFO server_args.py:2502: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=120 avail_mem=76.32 GB):   5%|▌         | 1/20 [00:00<00:03,  5.08it/s]

Capturing batches (bs=72 avail_mem=76.29 GB):  35%|███▌      | 7/20 [00:00<00:00, 19.40it/s]

Capturing batches (bs=24 avail_mem=76.26 GB):  65%|██████▌   | 13/20 [00:00<00:00, 23.24it/s]

Capturing batches (bs=2 avail_mem=72.64 GB):  80%|████████  | 16/20 [00:00<00:00, 21.86it/s]

Capturing batches (bs=1 avail_mem=72.64 GB): 100%|██████████| 20/20 [00:00<00:00, 21.44it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Maki. I'm a female college student from the United States. I attend Vassar College, and I'm majoring in Psychology, with a minor in English. I started college four years ago, and I'm very interested in politics. I'm planning on running for a vice president position in the American Psychological Association. I'm currently exploring different candidate groups to see which one is the most suitable for me. I have a job as a financial counselor for a large marketing firm in New York. I try to provide support to our clients with their financial needs, including creating financial plans, answering questions about financial products, and helping with
Prompt: The president of the United States is
Generated text:  trying to decide how many troops to have on a certain island. The president knows that there are 3 types of troops available: American, Canadian, and Indian. There are 12 American troops, 16 Canadian troops, and 14 Indian troops. The president

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I'm a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? I'm a [insert a unique trait or skill that sets me apart from others]. And what's your background? I've been working in [insert a specific field or industry] for [insert a number] years. And what's your favorite hobby or activity? I love [insert a hobby or activity that you enjoy]. And what's your favorite book or movie? I love [insert a favorite book or movie that you've read or watched]. And what

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris. It is the largest city in France and the second-largest city in the European Union. It is also the oldest capital city in Europe, having been founded in 789 AD. Paris is known for its rich history, art, and cuisine, and is a major tourist destination. It is also home to many famous landmarks such as the Eiffel Tower, the 

Generated text:  likely to be characterized by rapid advancements in several key areas, including:

1. Increased integration with human intelligence: AI is likely to become more integrated with human intelligence, allowing machines to learn and adapt to human behavior and preferences. This could lead to more personalized and efficient AI systems.

2. Enhanced machine learning capabilities: AI is likely to become more powerful and capable, with the ability to learn from large amounts of data and make more accurate predictions and decisions.

3. Greater use of AI in healthcare: AI is likely to play a more significant role in healthcare, with the ability to analyze medical data and provide more accurate diagnoses and treatment recommendations.





### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  Sarah, I'm a busy professional with a passion for photography and social media marketing. I'm currently in my mid-30s, and I love spending my free time exploring the world and connecting with people online. I'm a natural at crafting compelling content for both my social media accounts and my blog, and I'm always looking for new ways to expand my skills and network in my field. I'm excited to start my new adventure with you, and I look forward to learning more about you and your journey. How can I assist you today? What can I do for you today? What are your hobbies and interests? I'm

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris.
Paris is the capital and largest city of France and the largest metropolitan area in the European Union.
Paris is situated on the left bank of the Seine River, and on the outs

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

],

 and

...

 understand

 how

 AI

 systems

 arrive

In [6]:
llm.shutdown()